# Configuration

In [ ]:
# all directory path must have a '/' in the end

SOURCE_DIR='./sub_code/'  # location of the "sub_code"
indexDir='../multimodal/0_get_data/data/index_rand17/' # partition of image list
chooseDynamic=False # true: dynamic, false: static
outputDir="./data_extract_dynamic_static_features/" # where to save the output feature set

if chooseDynamic:
    # location of dynamic features
    feature_dir='./1_reproduce_dynamic_and_MKL/configuration_corrected/adjective_phase_set/'
else:
    # location of static features
    feature_dir='./1_reproduce_dynamic_and_MKL/static_features_original/'

In [ ]:
import sys
sys.path.append(SOURCE_DIR)

import os, pickle, utilities
import numpy as np

old_features = utilities.load_adjective_phase(feature_dir)
! mkdir {outputDir}    

In [ ]:
def loadLabels(indexDir):
    # structure of ans
    # ans[iadj][('train'+'validate'）/'test']['labels'/'image_ids']
    ans=[None]*24
    for i in range(24):
        ans[i]={}
        for aset in ['test', 'train', 'validate']:
            ans[i][aset]={}
            if i<6:
                iadj=i+1
            else:
                iadj=i+2
            
            fname=indexDir+str(iadj)+aset+".txt"
            lines=open(fname,"r").readlines()
            lines=[lines[j] for j in range(len(lines)) if (j%100)<8]
            lines=map(lambda line: line.split(' '), lines)
            
            # get image names and labels
            labels=map(lambda tpl: int(tpl[1].strip()), lines);
            labels=labels[0::8]
            # get image path -> pure file name -> id
            image_paths=map(lambda tpl: '_'.join(tpl[0].split('/')[-1].split("_")[:-2]), lines);
            image_paths=image_paths[0::8]
            
            if aset=='validate':
                ans[i]['train']['labels']=ans[i]['train']['labels']+labels
                ans[i]['train']['image_ids']=ans[i]['train']['image_ids']+image_paths
            else:
                ans[i][aset]['labels']=labels
                ans[i][aset]['image_ids']=image_paths
    return ans
dataset=loadLabels(indexDir);

In [ ]:
if chooseDynamic:
    Nfeature=16 # for dynamic
    outName=outputDir+"dynamic.pkl"
else:
    Nfeature=188 # for static
    outName=outputDir+"static.pkl"

import utilities
import copy
import pickle
import numpy as np

#fs=FS['smooth']['SLIDE_5CM']
def getObjFromFs(fs, objName, myLabel):
    ans={'label':[], 'feature':[]}
    for phase in ['train', 'test']:
        for i, obj in enumerate(fs[phase]['object_names']):
            if "_".join(obj.split("_")[:-2])==objName:
                assert(myLabel==fs[phase]['labels'][i])
                ans['label'] += [myLabel]
                ans['feature'] += [fs[phase]['features'][i]]
    return ans

def repartitionFeatures(fs, dataset):
    #FS['smooth']['SLIDE_5CM']['test']['object_names', 'labels', 'object_ids', 'features']
    #dataset[iadj]['train'/'test'][label / image_short_name]
    ans=[None]*24
    # ans[iadj]['train'/'test']['feature'/'label']
    for i in range(24):
        if utilities.adjectives[i]=="nice" or utilities.adjectives[i]=="slippery":
            print "skipping: ", utilities.adjectives[i]
            continue
        ans[i]={}
        for phase in ['train', 'test']:
            allObj={'feature':np.zeros((0, Nfeature)), 'label': np.zeros((0))}
            for idx, imName in enumerate(dataset[i][phase]['image_ids']):
                anObj={'feature':[], 'label':[]}
                for proc in fs['smooth'].keys():
                    temp=getObjFromFs(fs[utilities.adjectives[i]][proc], imName, dataset[i][phase]['labels'][idx])
                    if anObj['feature']==[]:
                        anObj=temp
                    else:
                        for il, label in enumerate(temp['label']):
                            assert(label==anObj['label'][il])
                        for il, feature in enumerate(temp['feature']):
                            anObj['feature'][il] = np.concatenate((anObj['feature'][il], feature))
                anObj['feature']=np.array(anObj['feature'])
                anObj['label']=np.array(anObj['label'])
                for prop in allObj.keys():
                    #print i, phase, idx, imName, prop, anObj[prop].shape
                    #print anObj
                    allObj[prop] = np.concatenate( (allObj[prop], anObj[prop]) )
            ans[i][phase]=allObj
    return ans

formatted=repartitionFeatures(old_features, dataset);
pickle.dump(formatted, open(outName,'w'))